## Import the necessary libraries
- pandas
- sqlite3
    - make the connection to the database of 'sql-murder-mystery.db'

In [1]:
import pandas as pd

In [2]:
import sqlite3
conn = sqlite3.connect('sql-murder-mystery.db')
cur = conn.cursor()

## From the given information:
- The crime was committed in SQL City on the date Jan. 18, 2015

In [3]:
cur.execute('''SELECT * 
               FROM crime_scene_report
               WHERE date = '20180215' AND city = 'SQL City';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,date,type,description,city
0,20180215,murder,REDACTED REDACTED REDACTED,SQL City
1,20180215,murder,Someone killed the guard! He took an arrow to ...,SQL City


## Checking the transcripts:
- I noticed the last two transcripts seemed suspicious
    - Zeroed in using df.tail(2)

In [4]:
cur.execute('''SELECT * 
               FROM interview
               WHERE transcript IS NOT '\n';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.tail(2)

,person_id,transcript
3736,16371,"I saw the murder happen, and I recognized the ..."
3737,67318,I was hired by a woman with a lot of money. I ...


## Investigating more:
- Wanted to see there were any other keywords in the transcripts with the work 'murder'
    - Came up with only three
        - The only one that seemed like it would lead anywhere was person_id: 16371

In [5]:
cur.execute('''SELECT * 
               FROM interview
               WHERE transcript LIKE '%murder%';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,person_id,transcript
0,14849,murder to leave it behind?’ She said the last ...
1,15121,"Queen jumped up and bawled out, “He’s murderin..."
2,16371,"I saw the murder happen, and I recognized the ..."


## Dive deeper into person_id:
- In order to see who this person_id belonged to, I went to the schema and saw that 'get_fit_now_member' table used person_id, just like the 'interview' table.  So, I joined them through their person_id in order to get the name.

In [6]:
cur.execute('''SELECT * 
               FROM interview i
               JOIN get_fit_now_member gm
               ON i.person_id = gm.person_id
               WHERE transcript LIKE '%murder%';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,person_id,transcript,id,person_id,name,membership_start_date,membership_status
0,16371,"I saw the murder happen, and I recognized the ...",90081,16371,Annabel Miller,20160208,gold


## Investigating more:
- This is the one that really stood out, because the transcript said "I was hired by a woman with alot of money..."  I've watched enough murder shows to know that's not a good sign!
    - I followed the same procedure to get the person_id of the person who said this.  Then joined to the 'get_fit_now_member' table to find the name.

In [7]:
cur.execute('''SELECT * 
               FROM interview
               WHERE transcript LIKE '%hire%';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,person_id,transcript
0,57234,"‘Cheshire Puss,’ she began, rather timidly, as..."
1,27707,was a little startled by seeing the Cheshire C...
2,70884,was a little startled by seeing the Cheshire C...
3,12329,‘It’s the Cheshire Cat: now I shall have someb...
4,41067,‘It’s the Cheshire Cat: now I shall have someb...
5,87608,‘I didn’t know that Cheshire cats always grinn...
6,85497,"‘It’s a friend of mine--a Cheshire Cat,’ said ..."
7,72661,"‘Cheshire Puss,’ she began, rather timidly, as..."
8,66991,"‘It’s a Cheshire cat,’ said the Duchess, ‘and ..."
9,61016,‘I didn’t know that Cheshire cats always grinn...


In [8]:
cur.execute('''SELECT * 
               FROM interview i
               JOIN get_fit_now_member gm
               ON i.person_id = gm.person_id
               WHERE transcript LIKE '%hire%';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,person_id,transcript,id,person_id,name,membership_start_date,membership_status
0,67318,I was hired by a woman with a lot of money. I ...,48Z55,67318,Jeremy Bowers,20160101,gold


## Checking to see if Jeremy Bowers is the murderer

In [9]:
cur.execute('''INSERT INTO solution VALUES (1, "Jeremy Bowers");''')
conn.commit()

In [10]:
cur.execute("""SELECT value FROM solution;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,value
0,"Congrats, you found the murderer! But wait, th..."


## Still need to find the woman that hired him.
- Possible suspects? 
    - People that checked in on the day of the murder?

In [19]:
cur.execute('''SELECT * 
               FROM get_fit_now_check_in c
               JOIN get_fit_now_member m
               ON c.membership_id = m.id
               WHERE check_in_date = '20180215';''')
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df

,membership_id,check_in_date,check_in_time,check_out_time,id,person_id,name,membership_start_date,membership_status
0,K055L,20180215,1056,1097,K055L,60785,Latisha Larzazs,20170503,regular
1,TXNSC,20180215,176,843,TXNSC,18044,Roberto Slot,20170510,silver
2,E18IN,20180215,756,1005,E18IN,64073,Lyda Mandigo,20180316,silver
3,3OOJ1,20180215,954,1050,3OOJ1,62367,Concha Fiveash,20180109,regular
4,X377L,20180215,465,694,X377L,94492,Alicia Bernson,20170722,silver
5,MN1KI,20180215,241,809,MN1KI,43801,Art Kunsman,20180118,silver
